### Please note:

The structure of this notebook is nearly identical to the notebook where I fine-tuned the model on persuasive essays. Please look at that notebook for quite detailed commentary. I add further commentary here sparingly.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
!pip install transformers
!pip install accelerate
!pip install peft
!pip install datasets
!pip install bitsandbytes

In [ ]:
import torch
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          BitsAndBytesConfig,
                          Trainer,
                          TrainingArguments)
from datasets import load_dataset
from peft import (LoraConfig,
                  PeftConfig,
                  PeftModel,
                  get_peft_model,
                  prepare_model_for_kbit_training)

In [ ]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)

In [1]:
# enable AWS functionalities
!pip install boto3
!pip install s3fs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
    Uninstalling fsspec-2023.6.0:
      Successfully uninstalled fsspec-2023.6.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.102
    Uninstalling botocore-1.34.102:
      Successfully uninstalled botocore-1.34.102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.

In [2]:
import pandas as pd
train_narrative_model_input = pd.read_csv('s3://698modeldata/train_narrative_model_input.csv')
eval_narrative_model_input = pd.read_csv('s3://698modeldata/eval_narrative_model_input.csv')
test_narrative_model_input = pd.read_csv('s3://698modeldata/test_narrative_model_input.csv')

In [ ]:
train_narrative_model_input['target_score'].describe() # Range is 0 to 6, so we need 7 labels

count    12832.000000
mean         2.556032
std          1.077313
min          0.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: target_score, dtype: float64

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
        base_model,
        num_labels=7,
        quantization_config=bnb_config,
        device_map='auto',
        trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=2,
    bias='lora_only',
    task_type='SEQ_CLS',
    target_modules=['q_proj', 'v_proj', 'k_proj']
)

model = get_peft_model(model, peft_config)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from torch.utils.data import Dataset
import torch.nn.functional as F

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, feature_columns, max_length=2000, max_score=7):  # max_length 2000
        self.tokenizer = tokenizer
        self.data = df
        self.feature_columns = feature_columns
        self.max_length = max_length

        # Truncate from the beginning if necessary (less critical)
        self.encodings = tokenizer(list(df['final_input']), truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt", truncation_strategy='only_first')

        self.features = torch.tensor(df[self.feature_columns].values, dtype=torch.float32)

        self.labels = torch.tensor(df['target_score'].tolist(), dtype=torch.long)
        self.labels = F.one_hot(self.labels, num_classes=max_score).float()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['features'] = self.features[idx]
        item['labels'] = self.labels[idx]
        return item

# List of encoded
feature_columns = ['score_type_rater1_trait1', 'score_type_rater1_trait2',
       'score_type_rater1_trait3', 'score_type_rater1_trait4',
       'score_type_rater1_trait5', 'score_type_rater1_trait6',
       'score_type_rater2_trait1', 'score_type_rater2_trait2',
       'score_type_rater2_trait3', 'score_type_rater2_trait4',
       'score_type_rater2_trait5', 'score_type_rater2_trait6']


In [ ]:
train_narrative_model_input.columns

Index(['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'rater2_domain1',
       'rater3_domain1', 'domain1_score', 'rater1_trait1', 'rater1_trait2',
       'rater1_trait3', 'rater1_trait4', 'rater1_trait5', 'rater1_trait6',
       'rater2_trait1', 'rater2_trait2', 'rater2_trait3', 'rater2_trait4',
       'rater2_trait5', 'rater2_trait6', 'rater3_trait1', 'rater3_trait2',
       'rater3_trait3', 'rater3_trait4', 'rater3_trait5', 'rater3_trait6',
       'grade_level', 'dale_chall_score', 'complexity_difference',
       'pos_proportions', 'dependency_proportions', 'character_count_scaled',
       'word_count_scaled', 'error_count_scaled', 'error_to_word_ratio_scaled',
       'target_score', 'actual_criteria', 'actual_range', 'final_input',
       'score_type_rater1_trait1', 'score_type_rater1_trait2',
       'score_type_rater1_trait3', 'score_type_rater1_trait4',
       'score_type_rater1_trait5', 'score_type_rater1_trait6',
       'score_type_rater2_trait1', 'score_type_rater2_trait2

In [ ]:
from torch.utils.data import DataLoader

# instances of the dataset for training, evaluation, and testing
train_dataset = CustomDataset(train_narrative_model_input, tokenizer, feature_columns)
eval_dataset = CustomDataset(eval_narrative_model_input, tokenizer, feature_columns)
test_dataset = CustomDataset(test_narrative_model_input, tokenizer, feature_columns)

# DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, pin_memory=True)
eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False, pin_memory=True)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
model_path = '/content/drive/MyDrive/DATA698/models/best_narrative_model'


In [ ]:
from transformers import Trainer, TrainingArguments

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Made batch size smaller
    gradient_accumulation_steps=4,  # accumulate gradients over 2 steps
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    report_to="none",
    fp16=True,  # mixed precision
)

# Initialize  Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train
trainer.train()

# Save model + tokenizer
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.223800,0.229552
2,0.175800,0.215909
3,0.152700,0.211377


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

('/content/drive/MyDrive/DATA698/models/best_narrative_model/tokenizer_config.json',
 '/content/drive/MyDrive/DATA698/models/best_narrative_model/special_tokens_map.json',
 '/content/drive/MyDrive/DATA698/models/best_narrative_model/tokenizer.json')

In [ ]:
class InferenceDataset(Dataset):
    def __init__(self, df, tokenizer, feature_columns, max_length=2000):
        self.tokenizer = tokenizer
        self.data = df
        self.feature_columns = feature_columns
        self.max_length = max_length
        self.essay_ids = df['essay_id'].tolist()

        # Tokenization
        self.encodings = tokenizer(list(df['final_input']), truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt", truncation_strategy='only_first')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['essay_id'] = self.essay_ids[idx]
        # Determine  active feature
        active_feature = next((f for f in self.feature_columns if self.data.iloc[idx][f] == 1), None)
        item['feature'] = active_feature
        return item

test_dataset = InferenceDataset(df=test_narrative_model_input, tokenizer=tokenizer, feature_columns=feature_columns)
test_item = test_dataset[0]
print(test_item.keys())  # Check keys again


dict_keys(['input_ids', 'attention_mask', 'essay_id', 'feature'])


<ipython-input-18-51d2a3cc82d0>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
def custom_collate(batch):
    collated_batch = {key: torch.stack([item[key] for item in batch]) for key in batch[0] if key in ['input_ids', 'attention_mask', 'labels']}
    # Non-tensor data
    for key in ['essay_id', 'feature']:
        collated_batch[key] = [item[key] for item in batch]
    return collated_batch

# Update DataLoader
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=2, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=2, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
           

In [ ]:
# go to eval mode
model.eval()
predictions_info = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch['input_ids'].to(device)
        outputs = model(inputs)
        predictions = outputs.logits.argmax(dim=-1)

        for i in range(len(predictions)):
            pred_info = {
                'essay_id': batch['essay_id'][i],
                'feature': batch['feature'][i],
                'prediction': predictions[i].item()
            }
            predictions_info.append(pred_info)

#convert to df
import pandas as pd
narrative_predictions_df = pd.DataFrame(predictions_info)
print(narrative_predictions_df)



<ipython-input-18-51d2a3cc82d0>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


      essay_id                   feature  prediction
0        18467  score_type_rater1_trait1           3
1        18467  score_type_rater1_trait2           3
2        18467  score_type_rater1_trait3           3
3        18467  score_type_rater1_trait4           3
4        18467  score_type_rater2_trait1           3
...        ...                       ...         ...
2747     21245  score_type_rater1_trait6           4
2748     21245  score_type_rater2_trait1           4
2749     21245  score_type_rater2_trait2           4
2750     21245  score_type_rater2_trait5           4
2751     21245  score_type_rater2_trait6           4

[2752 rows x 3 columns]


In [ ]:
save_path = '/content/drive/MyDrive/DATA698/models/predictions/narrative_predictions_df.csv'


In [ ]:
narrative_predictions_df.to_csv(save_path, index=False)


In [ ]:
def get_active_feature(row, feature_columns):
    for feature in feature_columns:
        if row[feature] == 1:
            return feature
    return None

test_narrative_model_input['feature'] = test_narrative_model_input.apply(lambda row: get_active_feature(row, feature_columns), axis=1)
merged_test = pd.merge(narrative_predictions_df, test_narrative_model_input, on=['essay_id', 'feature'], how='left')



In [ ]:
final_df = merged_test.loc[:, ['essay_id', 'feature', 'prediction', 'target_score']]
save_path = '/content/drive/MyDrive/DATA698/models/predictions/final_narrative_predictions_df.csv'
final_df.to_csv(save_path, index=False)

In [ ]:
final_df.head()

,essay_id,feature,prediction,target_score
0,18467,score_type_rater1_trait1,3,3.0
1,18467,score_type_rater1_trait2,3,2.0
2,18467,score_type_rater1_trait3,3,2.0
3,18467,score_type_rater1_trait4,3,2.0
4,18467,score_type_rater2_trait1,3,3.0


Because this model has the most labels, we can make predictions on the other two categories of essays (i.e. persuasive and source-dependent) using this model. The structure is the same.

In [ ]:
test_persuasive_model_input = pd.read_csv('s3://698modeldata/test_persuasive_model_input.csv')

persuasive_feature_columns = ['score_type_domain1_score', 'score_type_domain2_score',
       'score_type_rater1_domain1', 'score_type_rater2_domain1']

test_dataset = InferenceDataset(df=test_persuasive_model_input, tokenizer=tokenizer, feature_columns=persuasive_feature_columns)
test_item = test_dataset[0]
print(test_item.keys())  # Check keys again

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# go to eval mode
model.eval()
predictions_info = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch['input_ids'].to(device)
        outputs = model(inputs)
        predictions = outputs.logits.argmax(dim=-1)

        for i in range(len(predictions)):
            pred_info = {
                'essay_id': batch['essay_id'][i],
                'feature': batch['feature'][i],
                'prediction': predictions[i].item()
            }
            predictions_info.append(pred_info)

#convert to df
import pandas as pd
persuasive_predictions_using_narrative_df = pd.DataFrame(predictions_info)
print(persuasive_predictions_using_narrative_df)


<ipython-input-18-51d2a3cc82d0>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


dict_keys(['input_ids', 'attention_mask', 'essay_id', 'feature'])
      essay_id                    feature  prediction
0         3587   score_type_domain1_score           2
1         3587   score_type_domain2_score           2
2          926  score_type_rater1_domain1           3
3          926  score_type_rater2_domain1           3
4          648  score_type_rater1_domain1           3
...        ...                        ...         ...
1071       185  score_type_rater2_domain1           3
1072      1541  score_type_rater1_domain1           3
1073      1541  score_type_rater2_domain1           3
1074      3956   score_type_domain1_score           4
1075      3956   score_type_domain2_score           4

[1076 rows x 3 columns]


In [ ]:
save_path = '/content/drive/MyDrive/DATA698/models/predictions/special_predictions/persuasive_predictions_using_narrative_df.csv'

persuasive_predictions_using_narrative_df.to_csv(save_path, index=False)


In [ ]:
test_persuasive_model_input['feature'] = test_persuasive_model_input.apply(lambda row: get_active_feature(row, persuasive_feature_columns), axis=1)
merged_test = pd.merge(persuasive_predictions_using_narrative_df, test_persuasive_model_input, on=['essay_id', 'feature'], how='left')





In [ ]:
final_df = merged_test.loc[:, ['essay_id', 'feature', 'prediction', 'target_score']]

save_path = '/content/drive/MyDrive/DATA698/models/predictions/special_predictions/final_persuasive_predictions_using_narrative_df.csv'
final_df.to_csv(save_path, index=False)

In [ ]:
# and same for source_dependent

test_source_dependent_model_input = pd.read_csv('s3://698modeldata/test_source_dependent_model_input.csv')

sd_feature_columns = ['score_type_rater1_domain1', 'score_type_rater2_domain1']

test_dataset = InferenceDataset(df=test_source_dependent_model_input, tokenizer=tokenizer, feature_columns=sd_feature_columns)
test_item = test_dataset[0]
print(test_item.keys())  # Check keys again

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=custom_collate)

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# go to eval mode
model.eval()
predictions_info = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch['input_ids'].to(device)
        outputs = model(inputs)
        predictions = outputs.logits.argmax(dim=-1)

        for i in range(len(predictions)):
            pred_info = {
                'essay_id': batch['essay_id'][i],
                'feature': batch['feature'][i],
                'prediction': predictions[i].item()
            }
            predictions_info.append(pred_info)

#convert to df
import pandas as pd
sd_predictions_using_narrative_df = pd.DataFrame(predictions_info)
print(sd_predictions_using_narrative_df)


<ipython-input-18-51d2a3cc82d0>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


dict_keys(['input_ids', 'attention_mask', 'essay_id', 'feature'])
      essay_id                    feature  prediction
0        15783  score_type_rater1_domain1           3
1        15783  score_type_rater2_domain1           3
2         9977  score_type_rater1_domain1           3
3         9977  score_type_rater2_domain1           3
4        13140  score_type_rater1_domain1           3
...        ...                        ...         ...
2127      7116  score_type_rater2_domain1           2
2128     14948  score_type_rater1_domain1           4
2129     14948  score_type_rater2_domain1           4
2130     10474  score_type_rater1_domain1           3
2131     10474  score_type_rater2_domain1           3

[2132 rows x 3 columns]


In [ ]:
save_path = '/content/drive/MyDrive/DATA698/models/predictions/special_predictions/sd_predictions_using_narrative_df.csv'
sd_predictions_using_narrative_df.to_csv(save_path, index=False)


In [ ]:
test_source_dependent_model_input['feature'] = test_source_dependent_model_input.apply(lambda row: get_active_feature(row, sd_feature_columns), axis=1)
merged_test = pd.merge(sd_predictions_using_narrative_df, test_source_dependent_model_input, on=['essay_id', 'feature'], how='left')

final_df = merged_test.loc[:, ['essay_id', 'feature', 'prediction', 'target_score']]

save_path = '/content/drive/MyDrive/DATA698/models/predictions/special_predictions/final_sd_predictions_using_narrative_df.csv'
final_df.to_csv(save_path, index=False)

